## 🔄 Ajuste do Diretório de Trabalho

Antes de carregar ou manipular arquivos, é importante garantir que estamos no diretório correto do projeto.  
O código abaixo verifica se o notebook está sendo executado a partir da pasta `notebooks`. Se for o caso, ele sobe um nível na hierarquia de diretórios para garantir que o diretório de trabalho seja a raiz do projeto.

Isso é útil para manter caminhos relativos consistentes ao acessar dados, scripts ou outros recursos do projeto.

📌 **Resumo do que o código faz:**
- Verifica se o diretório atual termina com `notebooks`.
- Se sim, volta uma pasta (para a raiz do projeto).
- Exibe o novo diretório de trabalho.


In [1]:
import os

# Verifica se o diretório de trabalho atual termina com 'notebooks'
if os.path.basename(os.getcwd()) == 'notebooks':
    # Se sim, sobe um nível de diretório para a pasta raiz do projeto
    os.chdir('..')

# Imprime o diretório de trabalho para confirmar que a mudança foi feita
print(f"Diretório de Trabalho Atual: {os.getcwd()}")

Diretório de Trabalho Atual: c:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov


## 📦 Carregamento das Bibliotecas para a Fase 3: Modelagem Preditiva

Nesta etapa, carregamos todas as bibliotecas necessárias para realizar o treinamento, validação e interpretação de modelos de Machine Learning aplicados à previsão da tendência do Ibovespa.

---

### 🔧 Principais Componentes Importados:

#### 📊 Manipulação de Dados
- `pandas`, `numpy`: Estruturação e transformação de dados tabulares e numéricos.
- `duckdb`: Consulta e carregamento eficiente da base persistida na fase anterior.

#### ⚙️ Modelagem e Avaliação
- `lightgbm`: Framework de gradient boosting eficiente, usado para modelagem supervisionada.
- `sklearn.model_selection.train_test_split`: Divisão da base de forma temporal para simular previsão realista.
- `sklearn.metrics`: Avaliação com métricas como `accuracy`, `ROC AUC`, `confusion_matrix`.

#### 🧠 Interpretação do Modelo
- `shap`: Framework de interpretabilidade para entender a importância das features no modelo treinado.

#### 📈 Visualização
- `matplotlib`, `seaborn`: Criação de gráficos e análise visual dos resultados.

#### 🛠️ Configuração do Projeto
- `src.config`: Importa o caminho e demais parâmetros definidos nas fases anteriores.

---

✅ Todas as bibliotecas e dependências da **Fase 3 - Modelagem** foram carregadas com sucesso e estão prontas para uso.


In [8]:
import duckdb
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
import optuna
from sklearn.model_selection import train_test_split, TimeSeriesSplit # Usaremos para a divisão temporal
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns

# Importa nossas configurações de projeto
import src.config as config

# Helpers do Notebook
from IPython.display import display

# Configurações de estilo
sns.set_theme(style='whitegrid', palette='viridis')
plt.style.use("fivethirtyeight")
%matplotlib inline

print("✅ Bibliotecas para a Fase 3 carregadas com sucesso!")

✅ Bibliotecas para a Fase 3 carregadas com sucesso!


## 🧱 Carregamento dos Dados para Modelagem

Nesta etapa, buscamos no banco de dados DuckDB a tabela `features_completas`, que contém todas as variáveis criadas e tratadas na **Fase 2** do projeto. Esses dados são a base para o treinamento dos modelos de Machine Learning.

---

### 📥 Etapas Realizadas:

1. **Conexão ao Banco DuckDB**
   - Utilizamos o caminho salvo no módulo `config`.

2. **Leitura da Tabela `features_completas`**
   - A tabela contém os dados finais após a engenharia de atributos, com a variável `alvo` (target) já definida.

3. **Conversão da Coluna `data`**
   - A coluna `data` é convertida para o tipo `datetime` e definida como índice do DataFrame.
   - Essa configuração é fundamental para **garantir uma divisão temporal correta** entre treino e teste, evitando vazamento de dados.

---

✅ Ao final desta célula, temos o DataFrame `df_model` carregado, indexado por data e pronto para os próximos passos de preparação e modelagem.


In [3]:
print(f"Carregando dados da tabela 'features_completas' de: {config.DB_PATH}")

try:
    con = duckdb.connect(database=str(config.DB_PATH), read_only=True)
    # MUITO IMPORTANTE: Selecionar da nova tabela com todas as features
    df_model = con.execute("SELECT * FROM features_completas").fetchdf()
    con.close()

    # Configura a coluna 'data' como o índice para facilitar a divisão temporal
    df_model['data'] = pd.to_datetime(df_model['data'])
    df_model.set_index('data', inplace=True)

    print("\n✅ Dados para modelagem carregados com sucesso!")
    print("Estrutura do DataFrame:")
    df_model.info()

except Exception as e:
    print(f"❌ Ocorreu um erro ao carregar os dados: {e}")

Carregando dados da tabela 'features_completas' de: C:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov\data\mercados.duckdb

✅ Dados para modelagem carregados com sucesso!
Estrutura do DataFrame:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2975 entries, 2014-02-05 to 2025-07-03
Data columns (total 83 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   close_petroleo_brent             2975 non-null   float64
 1   close_petrobras                  2975 non-null   float64
 2   close_dolar                      2975 non-null   float64
 3   close_ibovespa                   2975 non-null   float64
 4   close_sp500                      2975 non-null   float64
 5   high_petroleo_brent              2975 non-null   float64
 6   high_petrobras                   2975 non-null   float64
 7   high_dolar                       2975 non-null   float64
 8   high_ibovespa                    2975 non-null   

## 🎯 Criação da Variável Alvo Multiclasse

Para refinar o problema e permitir abordagens mais sofisticadas de modelagem, transformamos o alvo binário em uma **variável multiclasse com 3 categorias**, baseada na magnitude do retorno do dia seguinte do Ibovespa.

---

### 🧪 Lógica Utilizada

- **Classe 1 – Alta Significativa**: Retorno > +0.5%
- **Classe -1 – Baixa Significativa**: Retorno < -0.5%
- **Classe 0 – Neutro**: Variação entre -0.5% e +0.5%

Essa abordagem permite que o modelo diferencie entre movimentos significativos de mercado e ruídos de variação diária, tornando a previsão mais realista para aplicações práticas como alocação de risco e decisões de trading.

---

### 🔍 Resultado

A distribuição das classes no conjunto de dados foi verificada, garantindo equilíbrio e viabilidade para modelagem multiclasse. A última linha (que teria alvo indefinido) foi removida para manter a integridade do dataset.


In [4]:
# O DataFrame 'df_model' foi carregado e tem a data como índice.

print("--- Refinando o Problema: Criação do Alvo Multiclasse ---")

# 1. Definimos o nosso threshold de significância.
# Um movimento de 0.5% (0.005) é um bom ponto de partida.
# Podemos ajustar este valor mais tarde se necessário.
threshold = 0.005

# 2. Calculamos o retorno do dia seguinte para o Ibovespa
retorno_futuro = df_model['close_ibovespa'].pct_change().shift(-1)

# 3. Criamos a nova coluna 'alvo_multiclasse' com a lógica de 3 classes
# Usamos np.where aninhado, que funciona como um "SE/SENÃOSE/SENÃO"
df_model['alvo_multiclasse'] = np.where(
    retorno_futuro > threshold,      # Condição 1: Se o retorno futuro for > 0.5%
    1,                               # Então, a classe é 1 (Alta Significativa)
    np.where(
        retorno_futuro < -threshold, # Condição 2: Se o retorno futuro for < -0.5%
        -1,                          # Então, a classe é -1 (Baixa Significativa)
        0                            # Caso contrário, a classe é 0 (Neutra)
    )
)

# 4. Removemos o último dia, que terá um NaN no alvo
df_model.dropna(subset=['alvo_multiclasse'], inplace=True)

# --- Verificação ---
print("\nDistribuição do nosso novo alvo multiclasse (em %):")
# Usamos value_counts para ver quantas amostras temos de cada classe
print(df_model['alvo_multiclasse'].value_counts(normalize=True).sort_index().map('{:.2%}'.format))

print("\nExibindo as últimas linhas com o novo alvo para validação manual:")
display(df_model[['close_ibovespa', 'alvo_multiclasse']].tail(10))

--- Refinando o Problema: Criação do Alvo Multiclasse ---

Distribuição do nosso novo alvo multiclasse (em %):
alvo_multiclasse
-1    29.58%
 0    36.64%
 1    33.78%
Name: proportion, dtype: object

Exibindo as últimas linhas com o novo alvo para validação manual:


,close_ibovespa,alvo_multiclasse
data,,
2025-06-20,137116.0,0
2025-06-23,136551.0,0
2025-06-24,137165.0,-1
2025-06-25,135767.0,1
2025-06-26,137114.0,0
2025-06-27,136866.0,1
2025-06-30,138855.0,0
2025-07-01,139549.0,0
2025-07-02,139051.0,1


## 📦 Preparação dos Dados para o Modelo Multiclasse

Agora que já temos nossa variável alvo multiclasse (`alvo_multiclasse`), preparamos os dados para a modelagem com três etapas fundamentais:

---

### 🔁 1. Separação entre Features (X) e Alvo (y)

- **X**: Todas as colunas do `df_model`, exceto `alvo_multiclasse` (nosso novo alvo) e `alvo` (alvo antigo binário, se ainda existir).
- **y**: Apenas a coluna `alvo_multiclasse`.

---

### ⏳ 2. Divisão Temporal: Treino vs. Teste

- Utilizamos **os últimos 6 meses como conjunto de teste**, mantendo a ordem temporal dos dados (fundamental para séries temporais financeiras).
- O restante do histórico é usado para treino do modelo.

---

### 📊 3. Verificação da Distribuição das Classes

- Avaliamos a proporção de cada classe (-1, 0, 1) tanto no treino quanto no teste.
- Isso nos ajuda a identificar possíveis desequilíbrios entre os conjuntos que possam afetar a performance do modelo.

---

Com isso, garantimos que os dados estão corretamente estruturados para prosseguir com a modelagem multiclasse do Ibovespa.


In [5]:
# O 'df_model' já contém a coluna 'alvo_multiclasse' que criamos.

print("--- Preparando Dados Finais para Modelo Multiclasse ---")

# 1. Separação de Features (X) e Alvo (y)
# X são todas as colunas, exceto nosso novo alvo. 
# Usamos errors='ignore' para não dar erro se a coluna 'alvo' antiga já foi removida.
X = df_model.drop(columns=['alvo_multiclasse', 'alvo'], errors='ignore') 

# y é apenas a nossa nova coluna alvo multiclasse.
y = df_model['alvo_multiclasse']

print("\nSeparação X/y concluída.")
print("Shape de X (features):", X.shape)
print("Shape de y (alvo):", y.shape)

# 2. Divisão Temporal em Treino e Teste
# Mantemos a mesma lógica: últimos 6 meses para teste.
ponto_de_corte = X.index.max() - pd.DateOffset(months=6)

print(f"\nData de corte para o conjunto de teste: {ponto_de_corte.date()}")

X_treino = X[X.index < ponto_de_corte]
X_teste = X[X.index >= ponto_de_corte]

y_treino = y.loc[X_treino.index]
y_teste = y.loc[X_teste.index]

print("\n--- Tamanho dos Conjuntos ---")
print(f"Conjunto de Treino: {len(X_treino)} amostras")
print(f"Conjunto de Teste: {len(X_teste)} amostras")

# 3. Verificação da distribuição do alvo nos dois conjuntos
print("\nDistribuição do alvo no conjunto de Treino:")
print(y_treino.value_counts(normalize=True).sort_index().map('{:.2%}'.format))

print("\nDistribuição do alvo no conjunto de Teste:")
print(y_teste.value_counts(normalize=True).sort_index().map('{:.2%}'.format))

--- Preparando Dados Finais para Modelo Multiclasse ---

Separação X/y concluída.
Shape de X (features): (2975, 82)
Shape de y (alvo): (2975,)

Data de corte para o conjunto de teste: 2025-01-03

--- Tamanho dos Conjuntos ---
Conjunto de Treino: 2846 amostras
Conjunto de Teste: 129 amostras

Distribuição do alvo no conjunto de Treino:
alvo_multiclasse
-1    29.94%
 0    36.05%
 1    34.01%
Name: proportion, dtype: object

Distribuição do alvo no conjunto de Teste:
alvo_multiclasse
-1    21.71%
 0    49.61%
 1    28.68%
Name: proportion, dtype: object


## 🔧 Otimização de Hiperparâmetros com Optuna (Modelo Multiclasse)

Nesta etapa, usamos a biblioteca **Optuna** para encontrar a melhor combinação de hiperparâmetros para o modelo **LightGBM Multiclasse**, maximizando a métrica **AUC (One-vs-Rest)** em uma validação cruzada temporal.

---

### 🧪 Estratégia Utilizada

- **Validação Cruzada Temporal (TimeSeriesSplit)**: Utilizamos 5 dobras (splits) respeitando a ordem temporal dos dados, fundamental em séries temporais.
- **Função Objetivo**: Para cada combinação de parâmetros, treinamos o modelo em múltiplos folds e retornamos a média do AUC.
- **Espaço de Busca**: Foram testados hiperparâmetros como número de árvores (`n_estimators`), taxa de aprendizado (`learning_rate`), profundidade máxima (`max_depth`), número de folhas (`num_leaves`) e regularizações L1/L2 (`reg_alpha`, `reg_lambda`).
- **Modelo**: `LightGBMClassifier` com `objective='multiclass'` e `metric='multi_logloss'`.

---

### 📈 Resultados

- **Total de Combinações Avaliadas**: 100
- **Melhor Score (AUC Médio)**: Apresentado ao final da execução
- **Melhores Hiperparâmetros**: São armazenados em `best_params`

---

Com essa otimização, aumentamos significativamente as chances do nosso modelo multiclasse alcançar maior performance e generalização no conjunto de teste real.


In [ ]:
# Usaremos os dados X_treino, y_treino que já separamos.

print("--- Iniciando Otimização de Hiperparâmetros com Optuna ---")

# 1. Definir a estratégia de Validação Cruzada para Séries Temporais
# Usaremos 5 "cortes" (splits) nos nossos dados de treino.
tscv = TimeSeriesSplit(n_splits=5)

# 2. Definir a função "objetivo" que o Optuna irá maximizar
def objective(trial):
    # Definimos o espaço de busca para os hiperparâmetros do LightGBM
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_class': 3, # Temos 3 classes: 1 (Alta), -1 (Baixa), 0 (Neutro)
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0), # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0), # L2 regularization
        'random_state': 42,
        'verbosity': -1,
        'n_jobs': -1 # Usa todos os cores da CPU
    }
    
    # Lista para guardar os scores de cada dobra da validação cruzada
    scores = []
    
    # Loop de Validação Cruzada
    for train_index, val_index in tscv.split(X_treino):
        X_train_fold, X_val_fold = X_treino.iloc[train_index], X_treino.iloc[val_index]
        y_train_fold, y_val_fold = y_treino.iloc[train_index], y_treino.iloc[val_index]
        
        # Treina o modelo
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train_fold, y_train_fold)
        
        # Faz previsões de probabilidade e calcula o score AUC
        y_proba = model.predict_proba(X_val_fold)
        score = roc_auc_score(y_val_fold, y_proba, multi_class='ovr')
        scores.append(score)
        
    # Retorna a média dos scores. O Optuna tentará maximizar este valor.
    return np.mean(scores)

# 3. Criar e executar o estudo
# 'direction="maximize"' porque queremos o maior AUC possível
study = optuna.create_study(direction="maximize")
# Vamos testar 100 combinações diferentes de parâmetros
study.optimize(objective, n_trials=100)

# 4. Exibir os resultados
print("\n--- Otimização Concluída ---")
print("Número de trials concluídos: ", len(study.trials))
print("Melhor score AUC na validação cruzada: ", study.best_value)

print("\nMelhores hiperparâmetros encontrados:")
best_params = study.best_params
print(best_params)

[I 2025-07-06 19:10:20,592] A new study created in memory with name: no-name-30a03f7b-8f4f-4cc9-9cbc-a91e1376e320


--- Iniciando Otimização de Hiperparâmetros com Optuna ---


[I 2025-07-06 19:10:29,424] Trial 0 finished with value: 0.5212060135271501 and parameters: {'n_estimators': 202, 'learning_rate': 0.29357502678256947, 'num_leaves': 250, 'max_depth': 8, 'reg_alpha': 0.4180995409037658, 'reg_lambda': 0.5432989244088593}. Best is trial 0 with value: 0.5212060135271501.
[I 2025-07-06 19:10:44,702] Trial 1 finished with value: 0.5181096097073736 and parameters: {'n_estimators': 882, 'learning_rate': 0.2805789885508433, 'num_leaves': 255, 'max_depth': 8, 'reg_alpha': 0.2351600152544845, 'reg_lambda': 0.5581450814876018}. Best is trial 0 with value: 0.5212060135271501.
[I 2025-07-06 19:11:14,658] Trial 2 finished with value: 0.5215192465048546 and parameters: {'n_estimators': 383, 'learning_rate': 0.06926025441511806, 'num_leaves': 129, 'max_depth': 10, 'reg_alpha': 0.8937184587552391, 'reg_lambda': 0.3391058954527302}. Best is trial 2 with value: 0.5215192465048546.
[I 2025-07-06 19:11:31,947] Trial 3 finished with value: 0.5180419418747786 and parameters: